dataset: Nasa_TurboEngine
1. 잔여수명(RUL)은 Section-0을 기준으로 하는 Section-1의 역순(Section-1의 최대값-Section-1), 기존의 방법은 Section-1을 사용해 모델을 훈련
  *  Section-1~4까지 제거를 하여, 센서데이터만 사용

2. 전처리 과정에서 section-0 RUL 상위 6개를 0으로 치환하는 문제
  * 이상을 더 빨리 감지하기 위함인 것으로 추측. 이러한 아이디라면 하위 6개가 더 합리적으로 생각됨.
  * 하지만 각 secton-0별 RUL의 편차가 크기 때문에 고정된 수치가 아닌 하위 n%를 유사한 방법으로 고려.

3. 기존 평가 방식의 문제 : 모델이 추론한 각 instance마다의 평가가 아닌 모델이 추론한 각 instance를 section-0기준으로한 각 group의 최대값(=각 section-0의 RUL)만을 측정.
  * -->anomaly detection task로 변환

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
train_file = "./sample_data/train_FD001.txt"
test_file = "./sample_data/test_FD001.txt"
RUL_file = "./sample_data/RUL_FD001.txt"

data = pd.read_csv(train_file,sep=" ",header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,NaN,NaN
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,NaN,NaN
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,NaN,NaN
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,NaN,NaN
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,NaN,NaN


In [3]:
data.drop(columns=[26,27],inplace=True)
columns = ["Section-{}".format(i)  for i in range(26)]
data.columns = columns

data.head()

,Section-0,Section-1,Section-2,Section-3,Section-4,Section-5,Section-6,Section-7,Section-8,Section-9,...,Section-16,Section-17,Section-18,Section-19,Section-20,Section-21,Section-22,Section-23,Section-24,Section-25
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [4]:
# Names
MachineID_name = ["Section-0"]
RUL_name = ["Section-1"]
OS_name = ["Section-{}".format(i) for i in range(2,5)]
Sensor_name = ["Section-{}".format(i) for i in range(5,26)]

# Data in pandas DataFrame
MachineID_data = data[MachineID_name]
RUL_data = data[RUL_name]
OS_data = data[OS_name]
Sensor_data = data[Sensor_name]

# Data in pandas Series
MachineID_series = data["Section-0"]
RUL_series = data["Section-1"]

In [5]:
grp = RUL_data.groupby(MachineID_series)
max_cycles = np.array([max(grp.get_group(i)["Section-1"]) for i in MachineID_series.unique()])
print("Max Life >> ",max(max_cycles))
print("Mean Life >> ",np.mean(max_cycles))
print("Min Life >> ",min(max_cycles))

Max Life >>  362
Mean Life >>  206.31
Min Life >>  128


In [6]:
data.drop(columns=["Section-0","Section-4", "Section-5", "Section-9", "Section-10",  "Section-14","Section-20","Section-22", "Section-23"] , inplace=True)
data.head()

,Section-1,Section-2,Section-3,Section-6,Section-7,Section-8,Section-11,Section-12,Section-13,Section-15,Section-16,Section-17,Section-18,Section-19,Section-21,Section-24,Section-25
0,1,-0.0007,-0.0004,641.82,1589.70,1400.60,554.36,2388.06,9046.19,47.47,521.66,2388.02,8138.62,8.4195,392,39.06,23.4190
1,2,0.0019,-0.0003,642.15,1591.82,1403.14,553.75,2388.04,9044.07,47.49,522.28,2388.07,8131.49,8.4318,392,39.00,23.4236
2,3,-0.0043,0.0003,642.35,1587.99,1404.20,554.26,2388.08,9052.94,47.27,522.42,2388.03,8133.23,8.4178,390,38.95,23.3442
3,4,0.0007,0.0000,642.35,1582.79,1401.87,554.45,2388.11,9049.48,47.13,522.86,2388.08,8133.83,8.3682,392,38.88,23.3739
4,5,-0.0019,-0.0002,642.37,1582.85,1406.22,554.00,2388.06,9055.15,47.28,522.19,2388.04,8133.80,8.4294,393,38.90,23.4044


In [7]:
from sklearn.preprocessing import QuantileTransformer , PowerTransformer, MinMaxScaler

In [8]:
gen = MinMaxScaler(feature_range=(0, 1))
data = gen.fit_transform(data)
data = pd.DataFrame(data)
data=np.nan_to_num(data)

pt = PowerTransformer()
data = pt.fit_transform(data)
data

array([[-1.84688137, -0.31975365, -1.3916867 , ..., -0.76399955,
         1.42665355,  1.24248381],
       [-1.82422371,  0.87202066, -1.03174892, ..., -0.76399955,
         1.03734696,  1.29226848],
       [-1.80166973, -1.9503069 ,  1.01385485, ..., -2.35059786,
         0.72414823,  0.465476  ],
       ...,
       [ 1.25582441,  0.18308976,  0.01311911, ...,  2.66650739,
        -1.90974435, -2.75272539],
       [ 1.26682846, -0.50209065,  1.01385485, ...,  1.14636389,
        -2.49938469, -1.91568423],
       [ 1.27779988, -1.45465135, -1.75831429, ...,  1.68933126,
        -2.19492018, -1.99819889]])

In [9]:
from sklearn.model_selection import train_test_split, cross_val_score

In [10]:
def RUL_df():
    rul_lst = [j  for i in MachineID_series.unique() for j in np.array(grp.get_group(i)[::-1]["Section-1"])]
    rul_col = pd.DataFrame({"rul":rul_lst})
    return rul_col


X_train = np.array(data)

y_train = np.array(RUL_df()).reshape(-1,1)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)


print(X_train.shape,y_train.shape)
X_train

(20631, 17) (20631, 1)


array([[-1.84688137, -0.31975365, -1.3916867 , ..., -0.76399955,
         1.42665355,  1.24248381],
       [-1.82422371,  0.87202066, -1.03174892, ..., -0.76399955,
         1.03734696,  1.29226848],
       [-1.80166973, -1.9503069 ,  1.01385485, ..., -2.35059786,
         0.72414823,  0.465476  ],
       ...,
       [ 1.25582441,  0.18308976,  0.01311911, ...,  2.66650739,
        -1.90974435, -2.75272539],
       [ 1.26682846, -0.50209065,  1.01385485, ...,  1.14636389,
        -2.49938469, -1.91568423],
       [ 1.27779988, -1.45465135, -1.75831429, ...,  1.68933126,
        -2.19492018, -1.99819889]])

In [11]:
y_train.shape

(20631, 1)

In [12]:
max_cycles

array([192, 287, 179, 189, 269, 188, 259, 150, 201, 222, 240, 170, 163,
       180, 207, 209, 276, 195, 158, 234, 195, 202, 168, 147, 230, 199,
       156, 165, 163, 194, 234, 191, 200, 195, 181, 158, 170, 194, 128,
       188, 216, 196, 207, 192, 158, 256, 214, 231, 215, 198, 213, 213,
       195, 257, 193, 275, 137, 147, 231, 172, 185, 180, 174, 283, 153,
       202, 313, 199, 362, 137, 208, 213, 213, 166, 229, 210, 154, 231,
       199, 185, 240, 214, 293, 267, 188, 278, 178, 213, 217, 154, 135,
       341, 155, 258, 283, 336, 202, 156, 185, 200])

In [13]:
anomaly_percent=10
i=0
start=0
group_y_train=[]
for _ in range(len(max_cycles)):
    end=max_cycles[i]+start
    group_y_train.append(y_train[start:end].reshape(-1).tolist())
    start=end
    i+=1

top_n_len=np.round(max_cycles*anomaly_percent/100).astype(int)



group_replaced_y_train=[]
for i in range(100):
#     group_replaced_y_train=group_replaced_y_train + [0 for _ in range(top_n_len[i])] + group_y_train[i][top_n_len[i]:]
    group_replaced_y_train=group_replaced_y_train + group_y_train[i][top_n_len[i]:] + [0 for _ in range(top_n_len[i])]


In [14]:
df_train=pd.DataFrame(data)
# df_train['target'] = y_train
df_train['target'] = group_replaced_y_train
df_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,target
0,-1.846881,-0.319754,-1.391687,-1.871061,-0.081738,-0.932431,1.165602,-0.463375,-1.241598,-0.180157,0.270006,-1.091345,-0.118842,-0.562870,-0.764000,1.426654,1.242484,173
1,-1.824224,0.872021,-1.031749,-1.079195,0.263809,-0.601489,0.381987,-0.783002,-1.473454,-0.100809,1.241006,-0.299444,-0.711710,-0.209418,-0.764000,1.037347,1.292268,172
2,-1.801670,-1.950307,1.013855,-0.632442,-0.370142,-0.468676,1.032163,-0.159131,-0.595354,-1.045084,1.477500,-0.925346,-0.556074,-0.613173,-2.350598,0.724148,0.465476,171
3,-1.779219,0.320582,0.013119,-0.632442,-1.306119,-0.764661,1.287388,0.270731,-0.910192,-1.740645,2.263380,-0.151844,-0.504130,-2.262630,-0.764000,0.302629,0.766684,170
4,-1.756870,-0.865918,-0.677944,-0.589021,-1.294771,-0.223683,0.694398,-0.463375,-0.410588,-0.998474,1.092373,-0.763292,-0.506706,-0.276973,-0.073644,0.421056,1.086014,169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,1.233718,-0.182819,-1.031749,1.543554,1.200045,1.940086,-1.970282,1.282952,0.313653,1.779560,-2.224517,2.029475,-0.196707,1.373418,2.194637,-1.695146,-2.513125,0
20627,1.244788,-0.729616,-1.758314,1.627183,2.096288,2.314491,-2.407067,1.727085,0.288926,1.698603,-2.057885,1.611157,-0.283143,1.763772,1.146364,-2.462563,-1.196759,0
20628,1.255824,0.183090,0.013119,1.425039,1.825290,1.904545,-2.348902,1.832507,0.336315,1.832728,-1.746163,1.824332,0.058218,2.730644,2.666507,-1.909744,-2.752725,0
20629,1.266828,-0.502091,1.013855,1.094574,2.195198,1.771953,-2.534263,1.935820,0.752875,2.560941,-2.066886,1.718778,-0.068856,2.259970,1.146364,-2.499385,-1.915684,0


In [15]:
df_train = df_train[df_train['target']!=0].reset_index(drop=True)
df_train=df_train.drop(['target'],axis=1)
df_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,-1.846881,-0.319754,-1.391687,-1.871061,-0.081738,-0.932431,1.165602,-0.463375,-1.241598,-0.180157,0.270006,-1.091345,-0.118842,-0.562870,-0.764000,1.426654,1.242484
1,-1.824224,0.872021,-1.031749,-1.079195,0.263809,-0.601489,0.381987,-0.783002,-1.473454,-0.100809,1.241006,-0.299444,-0.711710,-0.209418,-0.764000,1.037347,1.292268
2,-1.801670,-1.950307,1.013855,-0.632442,-0.370142,-0.468676,1.032163,-0.159131,-0.595354,-1.045084,1.477500,-0.925346,-0.556074,-0.613173,-2.350598,0.724148,0.465476
3,-1.779219,0.320582,0.013119,-0.632442,-1.306119,-0.764661,1.287388,0.270731,-0.910192,-1.740645,2.263380,-0.151844,-0.504130,-2.262630,-0.764000,0.302629,0.766684
4,-1.756870,-0.865918,-0.677944,-0.589021,-1.294771,-0.223683,0.694398,-0.463375,-0.410588,-0.998474,1.092373,-0.763292,-0.506706,-0.276973,-0.073644,0.421056,1.086014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18561,1.005152,-0.775068,-1.031749,1.270136,1.686678,1.335339,-0.811972,1.165947,0.266977,1.240741,-0.927572,1.275276,-0.070335,0.030007,1.689331,-1.825003,-1.640425
18562,1.016915,-0.502091,-1.758314,1.287491,0.485120,1.108958,-0.801671,0.799330,0.818386,1.832728,-1.469451,1.158754,0.188824,1.751354,1.146364,-1.190477,-2.114626
18563,1.028642,0.228904,-1.031749,0.638824,0.728660,1.448775,-1.605540,1.165947,0.244183,1.359531,-1.938505,0.918440,0.114021,1.194842,1.146364,-1.425467,-0.646256
18564,1.040333,0.918078,1.339054,1.076813,1.397271,1.556304,-1.631994,1.727085,0.521075,1.417810,-1.383629,1.275276,-0.121855,1.924975,1.689331,-1.093846,-1.123309


In [16]:
df_test = pd.read_csv(test_file, sep=" ",header=None)
df_test.drop(columns=[26,27],inplace=True)
df_test.columns = columns
df_test.head()

,Section-0,Section-1,Section-2,Section-3,Section-4,Section-5,Section-6,Section-7,Section-8,Section-9,...,Section-16,Section-17,Section-18,Section-19,Section-20,Section-21,Section-22,Section-23,Section-24,Section-25
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [17]:
df_rul = pd.read_csv(RUL_file, names=['rul'])
df_rul.head()

,rul
0,112
1,98
2,69
3,82
4,91


In [18]:
RUL_name = ["Section-1"]
RUL_data = df_test[RUL_name]
MachineID_series = df_test["Section-0"]
grp = RUL_data.groupby(MachineID_series)
max_cycles = np.array([max(grp.get_group(i)["Section-1"]) for i in MachineID_series.unique()])
# max_cycles[0] = max_cycles[0] - 6

In [19]:
df_test.drop(df_test[["Section-0", "Section-4", "Section-5",  "Section-9", "Section-10",  "Section-14", "Section-20", "Section-22","Section-23"]], axis=1 , inplace=True)
df_test.head()

,Section-1,Section-2,Section-3,Section-6,Section-7,Section-8,Section-11,Section-12,Section-13,Section-15,Section-16,Section-17,Section-18,Section-19,Section-21,Section-24,Section-25
0,1,0.0023,0.0003,643.02,1585.29,1398.21,553.90,2388.04,9050.17,47.20,521.72,2388.03,8125.55,8.4052,392,38.86,23.3735
1,2,-0.0027,-0.0003,641.71,1588.45,1395.42,554.85,2388.01,9054.42,47.50,522.16,2388.06,8139.62,8.3803,393,39.02,23.3916
2,3,0.0003,0.0001,642.46,1586.94,1401.34,554.11,2388.05,9056.96,47.50,521.97,2388.03,8130.10,8.4441,393,39.08,23.4166
3,4,0.0042,0.0000,642.44,1584.12,1406.42,554.07,2388.03,9045.29,47.28,521.38,2388.05,8132.90,8.3917,391,39.00,23.3737
4,5,0.0014,0.0000,642.51,1587.19,1401.92,554.16,2388.01,9044.55,47.31,522.15,2388.03,8129.54,8.4031,390,38.99,23.4130


In [20]:
gen = MinMaxScaler(feature_range=(0, 1))
df_test = gen.fit_transform(df_test)
df_test = pd.DataFrame(df_test)
#df_test = df_test.rolling(20).mean()
pt = PowerTransformer()
df_test = pt.fit_transform(df_test)
df_test=np.nan_to_num(df_test)

In [21]:
def RUL_df():
    rul_lst = [j  for i in MachineID_series.unique() for j in np.array(grp.get_group(i)[::-1]["Section-1"])]
    rul_col = pd.DataFrame({"rul":rul_lst})
    return rul_col


X_test = np.array(df_test)

y_test = np.array(RUL_df()).reshape(-1,1)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)


print(X_test.shape,y_test.shape)
X_test

(13096, 17) (13096, 1)


array([[-1.7938502 ,  1.04936462,  1.00459956, ..., -0.43585161,
        -0.26636316,  0.42059548],
       [-1.75983242, -1.22056415, -1.03515135, ...,  0.37849296,
         0.89958112,  0.64708667],
       [-1.72609272,  0.14086219,  0.34489676, ...,  0.37849296,
         1.36106153,  0.96601877],
       ...,
       [ 1.83603259, -0.26773967, -1.03515135, ...,  1.88097691,
        -1.83688401, -1.50936271],
       [ 1.84590292, -1.71931856,  0.34489676, ...,  1.88097691,
        -1.5901386 , -0.80455247],
       [ 1.85572407,  0.59502963,  1.00459956, ...,  2.57907106,
        -1.33736328, -1.7132478 ]])

In [22]:
# print(sum(max_cycles))
# count = 0
# for cycle_len in max_cycles:
#     for i in range(10):
#         y_test[count+i] = 0
#     count = count + cycle_len
# print(count)

i=0
start=0
group_y_test=[]
for _ in range(len(max_cycles)):
    end=max_cycles[i]+start
    group_y_test.append(y_test[start:end].reshape(-1).tolist())
    start=end
    i+=1

top_n_len=np.round(max_cycles*anomaly_percent/100).astype(int)

group_replaced_y_test=[]
for i in range(100):
#     group_replaced_y_test=group_replaced_y_test + [0 for _ in range(top_n_len[i])] + group_y_test[i][top_n_len[i]:]
    group_replaced_y_test=group_replaced_y_test + group_y_test[i][top_n_len[i]:] + [0 for _ in range(top_n_len[i])]

In [23]:
df_test = pd.DataFrame(X_test)
# df_test['attack'] = y_test
df_test['attack'] = group_replaced_y_test
df_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,attack
0,-1.793850,1.049365,1.004600,1.333227,-0.543820,-0.981745,0.157202,-0.501657,-0.734950,-1.135844,-0.112400,-0.698911,-1.521652,-0.692054,-0.435852,-0.266363,0.420595,28
1,-1.759832,-1.220564,-1.035151,-2.026229,0.095924,-1.467193,1.720949,-1.077473,-0.259251,0.487480,0.721475,-0.143646,0.182103,-1.649791,0.378493,0.899581,0.647087,27
2,-1.726093,0.140862,0.344897,-0.000094,-0.207106,-0.467327,0.483115,-0.317637,0.000826,0.487480,0.351776,-0.698911,-0.907151,0.657711,0.378493,1.361062,0.966019,26
3,-1.692628,1.913041,0.007977,-0.050265,-0.786268,0.307571,0.420181,-0.689532,-1.352331,-0.667106,-0.703805,-0.325176,-0.561267,-1.201106,-1.300958,0.748684,0.423078,25
4,-1.659435,0.640456,0.007977,0.124510,-0.156604,-0.375254,0.562352,-1.077473,-1.453359,-0.498585,0.701649,-0.698911,-0.979154,-0.769737,-2.225587,0.673786,0.919656,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,1.816143,2.231376,0.007977,1.822924,2.173171,1.574625,-0.560445,-0.881417,3.751023,1.353767,-1.767171,-1.288265,4.158266,1.520245,1.148940,-1.652391,-1.589975,0
13092,1.826113,-0.494678,-0.334082,1.779129,1.487011,2.325282,-0.822779,-0.317637,3.721800,0.956764,-1.234850,0.381065,4.091635,0.887575,1.880977,-2.136804,-0.723323,0
13093,1.836033,-0.267740,-1.035151,2.253302,1.010007,0.365717,-1.063189,0.713093,3.823968,0.819230,-1.030634,-0.510221,4.261843,1.068803,1.880977,-1.836884,-1.509363,0
13094,1.845903,-1.719319,0.344897,1.866578,1.356678,2.011041,-0.616408,0.039405,3.855726,1.002015,-0.786960,0.209360,4.342249,1.508112,1.880977,-1.590139,-0.804552,0


In [24]:
df_test[df_test['attack']==0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,attack
28,-0.936937,0.640456,0.344897,-1.343102,-0.164675,-0.998690,0.546454,0.212695,-1.242765,0.088446,1.188865,0.034494,-0.533863,-0.117190,-0.435852,0.899581,0.279846,0
29,-0.909642,-1.129855,1.327938,0.801560,-0.455494,-0.526578,0.467344,0.382663,-1.068938,-0.014851,-0.003286,0.549714,-0.340404,-0.480254,-1.300958,1.439254,0.841433,0
30,-0.882551,-0.267740,1.327938,0.297019,-1.401165,-0.864041,0.984611,0.212695,-0.055059,-0.956655,0.015040,-0.143646,-0.905875,-0.795751,0.378493,-0.611304,0.195369,0
75,-0.523584,-1.311265,0.007977,0.049888,0.503767,0.275421,0.081236,0.382663,-1.258645,0.487480,-0.237888,0.549714,-1.152790,0.742428,-0.435852,0.377871,-0.569442,0
76,-0.499317,1.185697,-0.681651,0.918939,0.266406,-0.414821,0.051025,0.713093,-1.344230,0.088446,-1.250293,0.549714,-1.073174,0.703407,1.148940,-0.125789,-1.233323,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,1.816143,2.231376,0.007977,1.822924,2.173171,1.574625,-0.560445,-0.881417,3.751023,1.353767,-1.767171,-1.288265,4.158266,1.520245,1.148940,-1.652391,-1.589975,0
13092,1.826113,-0.494678,-0.334082,1.779129,1.487011,2.325282,-0.822779,-0.317637,3.721800,0.956764,-1.234850,0.381065,4.091635,0.887575,1.880977,-2.136804,-0.723323,0
13093,1.836033,-0.267740,-1.035151,2.253302,1.010007,0.365717,-1.063189,0.713093,3.823968,0.819230,-1.030634,-0.510221,4.261843,1.068803,1.880977,-1.836884,-1.509363,0
13094,1.845903,-1.719319,0.344897,1.866578,1.356678,2.011041,-0.616408,0.039405,3.855726,1.002015,-0.786960,0.209360,4.342249,1.508112,1.880977,-1.590139,-0.804552,0


In [25]:
df_test['attack']=df_test['attack'].apply(lambda x: 1 if x==0 else 0)
df_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,attack
0,-1.793850,1.049365,1.004600,1.333227,-0.543820,-0.981745,0.157202,-0.501657,-0.734950,-1.135844,-0.112400,-0.698911,-1.521652,-0.692054,-0.435852,-0.266363,0.420595,0
1,-1.759832,-1.220564,-1.035151,-2.026229,0.095924,-1.467193,1.720949,-1.077473,-0.259251,0.487480,0.721475,-0.143646,0.182103,-1.649791,0.378493,0.899581,0.647087,0
2,-1.726093,0.140862,0.344897,-0.000094,-0.207106,-0.467327,0.483115,-0.317637,0.000826,0.487480,0.351776,-0.698911,-0.907151,0.657711,0.378493,1.361062,0.966019,0
3,-1.692628,1.913041,0.007977,-0.050265,-0.786268,0.307571,0.420181,-0.689532,-1.352331,-0.667106,-0.703805,-0.325176,-0.561267,-1.201106,-1.300958,0.748684,0.423078,0
4,-1.659435,0.640456,0.007977,0.124510,-0.156604,-0.375254,0.562352,-1.077473,-1.453359,-0.498585,0.701649,-0.698911,-0.979154,-0.769737,-2.225587,0.673786,0.919656,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,1.816143,2.231376,0.007977,1.822924,2.173171,1.574625,-0.560445,-0.881417,3.751023,1.353767,-1.767171,-1.288265,4.158266,1.520245,1.148940,-1.652391,-1.589975,1
13092,1.826113,-0.494678,-0.334082,1.779129,1.487011,2.325282,-0.822779,-0.317637,3.721800,0.956764,-1.234850,0.381065,4.091635,0.887575,1.880977,-2.136804,-0.723323,1
13093,1.836033,-0.267740,-1.035151,2.253302,1.010007,0.365717,-1.063189,0.713093,3.823968,0.819230,-1.030634,-0.510221,4.261843,1.068803,1.880977,-1.836884,-1.509363,1
13094,1.845903,-1.719319,0.344897,1.866578,1.356678,2.011041,-0.616408,0.039405,3.855726,1.002015,-0.786960,0.209360,4.342249,1.508112,1.880977,-1.590139,-0.804552,1


In [26]:
df_test['attack'].value_counts()

0    11787
1     1309
Name: attack, dtype: int64

In [28]:
df_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,-1.846881,-0.319754,-1.391687,-1.871061,-0.081738,-0.932431,1.165602,-0.463375,-1.241598,-0.180157,0.270006,-1.091345,-0.118842,-0.562870,-0.764000,1.426654,1.242484
1,-1.824224,0.872021,-1.031749,-1.079195,0.263809,-0.601489,0.381987,-0.783002,-1.473454,-0.100809,1.241006,-0.299444,-0.711710,-0.209418,-0.764000,1.037347,1.292268
2,-1.801670,-1.950307,1.013855,-0.632442,-0.370142,-0.468676,1.032163,-0.159131,-0.595354,-1.045084,1.477500,-0.925346,-0.556074,-0.613173,-2.350598,0.724148,0.465476
3,-1.779219,0.320582,0.013119,-0.632442,-1.306119,-0.764661,1.287388,0.270731,-0.910192,-1.740645,2.263380,-0.151844,-0.504130,-2.262630,-0.764000,0.302629,0.766684
4,-1.756870,-0.865918,-0.677944,-0.589021,-1.294771,-0.223683,0.694398,-0.463375,-0.410588,-0.998474,1.092373,-0.763292,-0.506706,-0.276973,-0.073644,0.421056,1.086014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18561,1.005152,-0.775068,-1.031749,1.270136,1.686678,1.335339,-0.811972,1.165947,0.266977,1.240741,-0.927572,1.275276,-0.070335,0.030007,1.689331,-1.825003,-1.640425
18562,1.016915,-0.502091,-1.758314,1.287491,0.485120,1.108958,-0.801671,0.799330,0.818386,1.832728,-1.469451,1.158754,0.188824,1.751354,1.146364,-1.190477,-2.114626
18563,1.028642,0.228904,-1.031749,0.638824,0.728660,1.448775,-1.605540,1.165947,0.244183,1.359531,-1.938505,0.918440,0.114021,1.194842,1.146364,-1.425467,-0.646256
18564,1.040333,0.918078,1.339054,1.076813,1.397271,1.556304,-1.631994,1.727085,0.521075,1.417810,-1.383629,1.275276,-0.121855,1.924975,1.689331,-1.093846,-1.123309


In [31]:
# df_train.to_csv('../data/skt/train.csv')
# df_test.to_csv('../data/skt/test.csv')

df_train.drop([0,1,2]).to_csv('./sample_data/train.csv')
df_test.drop([0,1,2]).to_csv('./sample_data/test.csv')